In [30]:
import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

In [31]:
import pandas as pd
import numpy as np
import scipy.misc
from sklearn.model_selection import train_test_split
import seaborn as sns
import time
import tensorflow as tf
import ron_gauss

In [32]:
df = pd.read_csv('synthetic.csv')

In [33]:
df.columns

Index(['Unnamed: 0', 'shift', 'pickup_community_area',
       'dropoff_community_area', 'payment_type', 'trip_day_of_week',
       'trip_hour_of_day', 'fare', 'tips', 'trip_total', 'trip_seconds',
       'trip_miles'],
      dtype='object')

In [34]:
drop = ['taxi_id', 'company_id', 'Unnamed: 0']
categorical = ["trip_day_of_week", "payment_type", "dropoff_community_area", "pickup_community_area", "shift"]
continuous = [x for x in df.columns if x not in (categorical + drop)]
print(continuous)

['trip_hour_of_day', 'fare', 'tips', 'trip_total', 'trip_seconds', 'trip_miles']


In [35]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [36]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

def baseline_model():
    model = Sequential()
    model.add(Dense(33, input_dim=11, activation='relu'))
    model.add(Dense(33, activation='relu'))
    model.add(Dense(1, activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [37]:
#Continuous variables
from sklearn import preprocessing
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import make_pipeline

features = set(df.columns)
for target_feature in continuous:
    print(target_feature)
    input_features = features - {target_feature}
    input_data = df[input_features]
    private_target = df[target_feature]

    X_train, X_test, y_train, y_test = train_test_split(input_data, private_target, random_state=42)      

    start_time = time.time()
    
    pipe = make_pipeline(StandardScaler(), KerasRegressor(build_fn=baseline_model, epochs=10, batch_size=20, verbose=1))
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    r_squared = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print("(time={}s) Regression + RON_Gauss, target_feature: {}, r^2: {}, mse: {}".format(time.time() - start_time, target_feature, r_squared, mse))



trip_hour_of_day
Epoch 1/10
601773/601773 [==============================] - 779s 1ms/step - loss: 48.2772 - accuracy: 0.0417
Epoch 2/10
601773/601773 [==============================] - 779s 1ms/step - loss: 47.9409 - accuracy: 0.0417
Epoch 3/10
601773/601773 [==============================] - 779s 1ms/step - loss: 47.9276 - accuracy: 0.0417
Epoch 4/10
601773/601773 [==============================] - 779s 1ms/step - loss: 47.9190 - accuracy: 0.0417
Epoch 5/10
601773/601773 [==============================] - 779s 1ms/step - loss: 47.9012 - accuracy: 0.0417
Epoch 6/10
601773/601773 [==============================] - 794s 1ms/step - loss: 47.9231 - accuracy: 0.0417
Epoch 7/10
601773/601773 [==============================] - 841s 1ms/step - loss: 47.9147 - accuracy: 0.0416
Epoch 8/10
601773/601773 [==============================] - 844s 1ms/step - loss: 47.9171 - accuracy: 0.0417
Epoch 9/10
601773/601773 [==============================] - 843s 1ms/step - loss: 47.8902 - accuracy: 0.0416
Ep

NameError: name 'r2_score' is not defined